In [1]:
%matplotlib inline

In [2]:
# Author: Yulou Zhou
#Weight loss function by commonality (and normalize)
#padded sequence
#for loop to test parameters
#for loop to get prev generated chord


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import csv
import glob
import os
import pandas as pd
import numpy as np
import csv
from random import shuffle
from MyDataset import MyDataset
torch.manual_seed(1)

updated


In [3]:
batch_size = 512
train_data = MyDataset('relativedata', prevchord = 6)
weights = train_data.stats()
data_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
c=0
for i in data_loader:
    print("p: ", i['p'])
    print("m: ", i['m'])
    #print("mask: ", i['mask'])
    print("l: ", i['l'])
    c += 10
    if c > 0: break

p:  tensor([[  0,   0,   0,  ...,  10,   3,   7],
        [  6,   4,   7,  ...,   2,   4,   7],
        [  1,   4,   7,  ...,   6,   4,   7],
        ...,
        [  7,   3,   7,  ...,   2,   4,   7],
        [  1,   4,   7,  ...,  11,   4,   7],
        [  8,   4,   7,  ...,   6,   4,   7]])
m:  tensor([[ 10,  10,  10,  ...,   0,   0,   0],
        [ 12,  12,  12,  ...,   0,   0,   0],
        [ 10,  10,  10,  ...,   0,   0,   0],
        ...,
        [  3,   3,   3,  ...,   0,   0,   0],
        [ 11,  11,  11,  ...,   0,   0,   0],
        [  6,   6,   6,  ...,   0,   0,   0]])
l:  tensor([[  5,   4,   7],
        [  6,   4,   7],
        [  8,   4,   7],
        ...,
        [  7,   3,   7],
        [ 11,   4,   7],
        [  8,   4,   7]])


In [4]:
def normalize(array):
    return [i / sum(array) for i in array]
newweights = []

def weightloss(weights, eps):
    for w in weights:
        newweights.append(torch.FloatTensor( normalize([1/(i + eps) for i in w] ) ) )
    #print(newweights)


In [11]:
class LSTMTagger(nn.Module):

    def __init__(self, p_embed_dim, m_embed_dim, p_hidden_dim, m_hidden_dim, vocab_size, tagset_size, batch_size = 1):
        super(LSTMTagger, self).__init__()
        self.batch_size = batch_size
        
        self.p_embed_dim, self.m_embed_dim = p_embed_dim, m_embed_dim
        #refer to https://discuss.pytorch.org/t/can-we-use-pre-trained-word-embeddings-for-weight-initialization-in-nn-embedding/1222/2
        
        self.p_embed = nn.Embedding(vocab_size, p_embed_dim)
        self.m_embed = nn.Embedding(vocab_size, m_embed_dim)
        self.p_embed.weight.data.copy_(torch.from_numpy(np.identity(vocab_size)))
        self.m_embed.weight.data.copy_(torch.from_numpy(np.identity(vocab_size)))
        
        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.p_lstm = nn.LSTM(p_embed_dim, p_hidden_dim, bidirectional = True)
        self.m_lstm = nn.LSTM(m_embed_dim, m_hidden_dim, bidirectional = True)
        

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear((p_hidden_dim + m_hidden_dim) * 2, tagset_size)
        self.root2second = nn.Linear((p_hidden_dim + m_hidden_dim) * 2 + 1, tagset_size)
        self.root2third = nn.Linear((p_hidden_dim + m_hidden_dim) * 2 + 1, tagset_size)
        
        self.p_hidden = self.init_hidden(p_hidden_dim)
        self.m_hidden = self.init_hidden(m_hidden_dim)

    def init_one_hot(self, vocab_size):
        #initialize each embedding
        #stack them together (or other ways to have  pretrained embeddings)
        #pretrained_weight is a numpy matrix of shape (num_embeddings, embedding_dim)
        #we should turn data into torch.from_numpy(pretrained_weight)
        #embed.weight.data.copy_(torch.from_numpy(pretrained_weight))
        
        torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
        
    
    def init_hidden(self, hidden_dim):
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (torch.zeros(2, self.batch_size, hidden_dim),
                torch.zeros(2, self.batch_size, hidden_dim))

    def forward(self, prev, melody, mask):
        
        prev_embeds = self.p_embed(prev)
        x1 = torch.transpose(prev_embeds, 0, 1) #TODO: DEBUG THIS LINE (probably the last batch)
        
        #TODO: ensure that melody input is a tensor
        
        melody_embeds =  self.m_embed(melody) #Error here
        #print(melody_embeds.size())
        x2 = torch.transpose(melody_embeds, 0, 1)[:8]
        
        p_lstm_out, self.p_hidden = self.p_lstm(x1, self.p_hidden)
        m_lstm_out, self.m_hidden = self.m_lstm(x2, self.m_hidden)      
        
        p_fstate, m_fstate = p_lstm_out[-1], m_lstm_out[-1]
        

        concat = torch.cat((p_fstate, m_fstate), 1)
        #print(p_fstate.size(), m_fstate.size())
        tag_space = self.hidden2tag(concat) #REASON: you only need the final state
        tag_scores = F.log_softmax(tag_space, dim = 1)
        

        withroot = torch.cat((concat, tag_scores.max(1)[1].float().view(-1,1)), 1)
        second = self.root2second(withroot)
        third = self.root2third(withroot)
        second_scores = F.log_softmax(second, dim=1)
        third_scores = F.log_softmax(third, dim=1)
        
        #stacked = torch.stack([tag_scores, second_scores, third_scores], dim=0)

        return [tag_scores, second_scores, third_scores]

In [55]:
import random
for iteration in range(1):

    P_HIDDEN_DIM = max(int(np.random.normal(25, 30)), 5)
    M_HIDDEN_DIM = max(int(np.random.normal(35, 30)), 5)
    vocab_size = 13
    tagset_size = 13
    EPS = random.uniform(0.005, 0.15)
    ITER = 10000
    P_EMBEDDING_DIM = vocab_size
    M_EMBEDDING_DIM = vocab_size
    P_HIDDEN_DIM = 18
    M_HIDDEN_DIM = 10
    EPS = 0.02806

    print("iter: ", ITER, "  phid: ", P_HIDDEN_DIM, "  mhid: ", M_HIDDEN_DIM, "  eps: ", EPS)
    newweishts = weightloss(weights, 0.005)
    model = LSTMTagger(P_EMBEDDING_DIM, M_EMBEDDING_DIM, P_HIDDEN_DIM, M_HIDDEN_DIM, vocab_size, tagset_size, batch_size = batch_size)
    loss_function = nn.NLLLoss(weight = newweights[0])
    loss_function2 = nn.NLLLoss(weight = (newweights[1]))
    loss_function3 = nn.NLLLoss(weight = (newweights[2]))
    optimizer = optim.SGD(model.parameters(), lr=0.2)
    
    for epoch in range(ITER):  # again, normally you would NOT do 300 epochs, it is toy data
        for i in data_loader:
            # Step 1. Remember that Pytorch accumulates gradients.
            # We need to clear them out before each instance
            model.zero_grad()

            # Also, we need to clear out the hidden state of the LSTM,
            # detaching it from its history on the last instance.
            model.p_hidden = model.init_hidden(P_HIDDEN_DIM)
            model.m_hidden = model.init_hidden(M_HIDDEN_DIM)

            # Step 2. Get our inputs ready for the network, that is, turn them into
            # Tensors of word indices.
            p, m, l, mask = i['p'], i['m'], i['l'], i['mask']
            #print(m)
            tag_scores = model(p, m, mask)
            # Step 3. Run our forward pass.

            # Step 4. Compute the loss, gradients, and update the parameters by
            #  calling optimizer.step()
            #print(targets[:1])
            loss = 0

            loss += loss_function(tag_scores[0], torch.t(l)[0])  
            loss += 0.5 * loss_function2(tag_scores[1], torch.t(l)[1])
            loss += 0.5 * loss_function3(tag_scores[2], torch.t(l)[2])   

            loss.backward()
            optimizer.step()
            break
        if(epoch%100 == 0): 
            print(epoch, loss)
            accuracy = 0.0
            c = 0
            for j in range(2):
                for i in data_loader:
                    c+=1
                    p, m, l, mask = i['p'], i['m'], i['l'], i['mask']
                    tag_scores = model(p, m, mask)
                    pred = tag_scores[0].max(1)[1].numpy()
                    truth = torch.t(l)[0].numpy()
                    print(pred[:20], truth[:20])
                    accuracy += np.average(pred == truth)
                    if (c>=1): break
            print("accuracy:", accuracy/c)

    accuracy = 0.0
    c = 0

    for j in range(10):
        for i in data_loader:
            c+=1
            p, m, l, mask = i['p'], i['m'], i['l'], i['mask']
            tag_scores = model(p, m, mask)
            pred = tag_scores[0].max(1)[1].numpy()
            truth = torch.t(l)[0].numpy()
            if(c<4): print(pred[:20], truth[:20])
            accuracy += np.average(pred == truth)
            if (c>=1): break
    print("accuracy:", accuracy/c)

iter:  10000   phid:  18   mhid:  10   eps:  0.02806
0 tensor(5.4374)
[7 7 8 5 7 7 7 7 7 7 8 7 7 7 7 7 8 7 7 7] [ 7  8  1  9  3  6  3 10 11  3  1  4  3  8  8 10  1  6 10  2]
[7 7 7 7 8 7 7 8 7 7 7 7 7 7 7 7 7 7 7 7] [ 3  1  8 10  3  1  1  1  3 10  7  6  3  8  1  6  1  3  1  8]
accuracy: 0.025390625
100 tensor(3.7759)
[6 6 6 6 8 8 8 6 8 8 8 6 6 6 6 8 8 6 6 8] [ 4  1 11  8  1  1  8  8  3  5  3  1  3  6  5  8 10  8  4  8]
[1 1 6 1 6 6 6 1 6 8 6 8 6 8 6 8 1 8 6 6] [10  1 11  8  1  8  1  6  6  7  1  6  1 12 11  1  1 10  1  3]
accuracy: 0.1904296875
200 tensor(3.5928)
[1 1 6 1 6 6 6 8 6 6 1 6 6 6 6 6 6 8 6 1] [ 1  1 12  6  9  6  8  8 11  1  8  8  8  1 10 10  9 10  3 12]
[6 6 1 6 6 1 1 6 1 8 1 6 6 6 1 6 6 6 1 8] [10  8  1  4  8 11  8  9  4  3  8 11 10  3  6 10  3 10  8  6]
accuracy: 0.1708984375
300 tensor(3.6221)
[6 6 8 8 6 6 8 8 6 8 8 8 8 0 8 8 8 8 8 8] [ 6  2  8  3  4  8  1 10  5  8  3  1  3  8  6  3  8  6  3  6]
[ 8  8  8  6  8  6  6  6  8  8  6  8  8  6  8  8 11  8  8  8] [ 8  3  5  7 10

2900 tensor(2.8114)
[ 8  5  4  1  6  8  0  3  0 12 10  1  5  9  6  8  5  4  3  3] [ 8  5 11  1  3 10  4  8  6 12 10  1 12  4  6 10 10  4 10 10]
[ 8 11 11  1 10  2  1  1  1  5  3  8  5  6  8  8  0 11  3  1] [ 4 11 11  3  6  2  8  8  3  5  3  5  8  4  8  8  8  6  1  9]
accuracy: 0.3037109375
3000 tensor(2.8014)
[ 1 10  6  6  8  5  6  6 10  3  9  6  3  3  2  6  8 10  8  6] [ 1  6  8  6  3 12  6  8 10  8  9  6  1  3  9  1  1  3  3  3]
[ 6 11 10 10  6 11  1  6 11 10  6  2  5  6  6  1  1 10  1 11] [ 3  6  1 10  1  6  1  1  9 10  6  8 12  1 10  8  6  6  7  4]
accuracy: 0.2958984375
3100 tensor(3.1264)
[ 5 11  5  7 11  3 10  3  1  1  1 11  6 11  1 11 10  1  3 10] [ 2  6  1  5 11  6 12  8  8  8  1  6  1  8  2  6 10  8  1  7]
[ 1  6  5 10 10  5  5  0  8  4  8  1  6  4  4 11  0  6  1  2] [ 8  3 10  8 10  5 10  3  3  9  3  1 11  4  1  6  1 11  1  6]
accuracy: 0.2802734375
3200 tensor(2.7762)
[10  4  1  5  6  1  6  1  1  1  6  6 11  8 11  6  1  8  1  1] [ 3  9 10  8  8  3  8 11  1  8 11  1 11 12  4

[ 1  4  6  3  6  8 10  7  6 12 11  1  8  6  1  1  3  6 10  0] [ 1  9  8  3 11  1 11  9  6  5  1  1  3 11  1  6  5 11 10  0]
accuracy: 0.3134765625
5800 tensor(2.8075)
[ 1  6  9  8 12  1  6  2  9 12  8  6  8  6  1  5  9  4 10 10] [10  6 12  8  5  1  8  9  6  8  3  1  3  6  1  3  4  4  5 10]
[ 2  1  3  1  8  1  6  4  3 10  5  7 10  1  6  4  4  8 10  2] [12  3  8  1  3  0  8  2 10  8  1  7  8  8  3  1 11  8  8  4]
accuracy: 0.3310546875
5900 tensor(2.7979)
[ 0  3  3  3 10  1  3 11  8  9  1  1  1  8  8 11  3  2 12 10] [10  8  3  3  1  1  8 11  3  9  1  5  6 10  3  1  8  9  7  6]
[ 1  8 11  4  3  6  0  5  0  1 11  1  1 10  8  5 11  3 11 11] [ 8  4  4  4  1  3  5  5  1  8  8  8  1 10  1  5  6 10  6  1]
accuracy: 0.296875
6000 tensor(2.9932)
[ 1  9  1  1 11 11  2  1  8  4  6  1  1  1  8 10  1  6  6  6] [ 1  1  1 10  6  8  9  8  8  6  8  3  3 10  6  3  3  1  1  1]
[ 6  6  4  8  8  1 10  4  8  1  2  6 11  8  6 10 11  3  5  1] [11 11  4  8  8  1 12  4  1  8  9  6  1  5  8  3  3  3 12  1]
accurac

8600 tensor(2.7940)
[10  8 11 11  2  1  3  6  2  1  8 11  1 11  6  6  8  5  8 12] [ 5  3  4  1  9  0  8  6  8  1  3  8  1  8 11  8  8  5  8 10]
[ 4 11  6 10  8 10 12  1  6  1  8  6  1  1  6  6  8 11 10  7] [ 4  1  6  3 10 10  6  3  6  1  6  8  3  4  6  6  8  8 10  4]
accuracy: 0.314453125
8700 tensor(2.7795)
[ 6  3  6  8  1  8  1 11  7  8  8  6  8 10  1  8  8  8  6  3] [ 8 10  1  4  3  1  1  9  8  5  8  5  8 10  1  8  8  8  6 10]
[ 3  6  6  4  4  6  4  9  3 12  6  3  4 10  5  2  8  5  1  5] [ 3 11  6  4  4 11  4 11  1  3  3  3  8  5  3 11  1  3  8  6]
accuracy: 0.3330078125
8800 tensor(2.6917)
[ 2  8 11  1  8 10  8  5  5  4 10  6  8  6  1  5  6  6  1  2] [ 3  8  4  8  1  8  8  7 10 11  3 10  5  1  1  5  1  1  1  2]
[10  1  5  1 10  6  4  0  6  8  8  2  8  1  5  5  5 11  2  8] [ 8  8 12  1  7  1 11  1  1  3  1 11  1  8  1 10  5 11  2  1]
accuracy: 0.32421875
8900 tensor(2.8389)
[ 1 11 12  3  6  1  2  0  3  4  7  4 10  1  1  9  1  8  6  1] [ 4 10  3 10  6  8  2  2  3  4 10  6  6  1  5  9

In [64]:

accuracy = 0.0
c = 0
for j in range(20):
    for i in data_loader:
        c+=1
        p, m, l, mask = i['p'], i['m'], i['l'], i['mask']
        tag_scores = model(p, m, mask)
        pred = tag_scores[0].max(1)[1].numpy()
        truth = torch.t(l)[0].numpy()
        print(pred[:20], truth[:20])
        accuracy += np.average(pred==truth)
        if (c>=1): break
print("accuracy:", accuracy/c)

[ 1  6 10  5  8  3  6  6  8 10 11 11  8 10  1  7  0  6  8  8] [10  1 10 10  8  3 10  1  3  1 11  6  8 10  6 11 11  3  8  8]
[ 1 10  6  2  2  3  5 10  8  6  6  1  6 11  1  6  8  8  6  6] [ 3  3  1  1 11  1  3  3  7  5 11  6  6  6  5 12  1  8  6 11]
[ 8  5  1 10  8  1 12  5  5  4  9  6  1  1  8 10  5 12  1  3] [ 8 10  6 12 10  6  9  1 10  4  9  8  6  1  8  3 10  8  9 10]
[ 6 10  8  7  8  8  3  8  8  6  1 11  4  6  2  2  1 10  6  8] [ 1  3  8 10  8  3 10  8  6 11  7  6  4  6  3 11  6  5 11  3]
[ 1 10 10  5  8  4  4  6  1  5  1 10  6  8  7  1  4  8 12 10] [ 8  3  1  1  3  8  8  8  8  4  3  5 11  8 11  1  4  1  9  8]
[ 9  4  1 10 10  1  5  4  1  6  0  2 11  9 10  3 10  8  1  5] [11  4  6  3  8  1  5 11  1  3  6  1 11  9  6 10  1  8  1 12]
[11  5  5  3 10  4  8  1  2  6 11 12 11  6 10  8  6  6  4 11] [ 4  6  5 12 10  3  3  8  9  6  1 10  4 11 10  8  6  1  6  1]
[ 4  1  8  3  4  6 10 11 10  6 11  2 10  8  1  7  8  5 11  1] [ 8  1  3  8 11  4  3  6  3 11 11  1 10  6  6  8  1 10  6  9]
[ 6 11  

In [67]:
torch.save(model, "/mnt/d/230/Iterations-blstm")
the_model = torch.load("/mnt/d/230/Iterations-blstm")

accuracy = 0.0
c = 0
for j in range(20):
    for i in data_loader:
        c+=1
        p, m, l, mask = i['p'], i['m'], i['l'], i['mask']
        tag_scores = the_model(p, m, mask)
        pred = tag_scores[0].max(1)[1].numpy()
        truth = torch.t(l)[0].numpy()
        print(pred[:20], truth[:20])
        accuracy += np.average(pred==truth)
        if (c>=1): break
print("accuracy:", accuracy/c)

/home/yuloucn/anaconda3/envs/nlu/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type LSTMTagger. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


[ 3  4  1 12  2  1  3  3  0  5  8  2  6  1 10 10  4  3  1  6] [ 6  4  1 12  4  1 10  4  0  5  8  4  1  1  3  3  4  8  1  1]
[ 3  1  4  5  2  6 11 11 11 10 11  8  6  8  6 10  1  2  7  1] [ 1  1 11  9  2 11  6  1  1 10 11  3  6 11  6  1  3  1  6  8]
[ 1  8  4 12  2  1  2  1  6  8  8  8  3  1  8  4  6  8 11  1] [ 1  3  9  4  9  1  2  6  6  1 12  1  6  1  8 11  8  1 11  1]
[ 8  6  0 10  2  6 10  5 11 10  8  4  1  4 10  3  1 11  5  8] [ 8  6  8  1  5  2  3  8  4 10  3  9  8 11 11  3  6  6  5  1]
[11  6  6 10  3 11  8  4  1  6  8  4  1  2  6  6 11 10  1  1] [ 6  1  6  8  3  6  4  1  1  8  8  3  6  9  1  8  4 10  5  9]
[ 1  5  8 10  8  1  8  4  4  5  5  6 10  4  6  6 11  8  6  4] [ 8  5  8  3  8  1  3 11  9 12  3 11 12 11  1 11 11  8  8  8]
[ 4  6  6  4  3  6 11  6 11  4 12  8 10  8  1  3 10  8  4 11] [ 4  6  6  4  3  6  7 11  9 11  2  8  3  8  1 10  3  1  6  1]
[ 6  2  3 11 10  5  2  9  1  3 10  3  6 11  1 12  3  6  3  4] [ 1  6 12  8 10  5  7 11  1 12  3  3  6 11  1  2  3 10  8 11]
[10  2  

KeyboardInterrupt: 

Create the model:



Train the model:

